## 1. Import libraries
To run some of the tools offered by Python we need to import libraries.

In [ ]:
# Library for scientific computing
import numpy as np
import pandas as pd
import math
# Libraries for random K-field generator
import sys
sys.path.append("C:\\Users\\Andrea\\Documents\\PT\\KFields_Generation")

import hydrogen as hg
# Libraries for flow simulation 
import flopy
# Libraries for contamiant transport simulation 
import yaml
import os
#save the current directory to restore it
#current_directory = os.getcwd()

import subprocess

current_directory = os.getcwd()

from subprocess import call
# Libraries for postprocessing data and UQ and R Analysis
import sys
sys.path.append("C:\\Users\\Andrea\\Documents\\PT")
import RAUQ_function as plib #change the name!!!!
#Library for visualization 
import matplotlib.pyplot as plt

import csv

## 2. Model domain & Grid definition 


In this section we define how to discretize the model domain of size $L_x \times L_y \times L_z$. $\Delta_C$, $\Delta_R$ and $\Delta_L$ represents the grid cell discretization. Here you also define the coordinates of the contaminant injection area of the target area and observation wells.   

In [ ]:
# Model domanin dimentions 
Lx = 230
Ly = 128
Lz = 1


# Grid 
ncol = 230 # number of colum of the grid
nrow = 128 # number of rows of the grid
nlay = 1 # number of layers of the grid 

# Grid cell dimentions 
del_R = Lx/ncol 
del_C = Ly/nrow
del_L = Lz/nlay

# Contaminant source area coordinates, where (0,0) is bottom left corner:
source_xl = 0 
source_xu = 0
source_yl = 0
source_yu = Ly


# control plane x coordinate
xPlane = 224

# Target area coordinates, where (0,0) is bottom left corner:
target_xl = xPlane
target_xu = xPlane
target_yl = 0
target_yu = Ly




# Observations wells coordinates, where (0,0) is bottom left corner:
observation_wells = [ (117, 55), (117, 75), (117, 95) ]


#Number of simulations of the Monte Carlo analysis
N_mc = 1000

## 3. Hydraulic Conductivity Fields Generation 
### Hydro_gen 

$\text{Hydro_gen}$ is a random ($Y \equiv \ln (K)$) field generator, where values of $ln(K)$ are spatially-correlated.
    
**WHAT YOU NEED**: 
    
-1 on MAC or LINUX: To run the following kernel you need to include the $\text{Hydro_gen}$ executable (hydrogen_linux or hydrogen_mac, accordingly to the used platform), the hydrogen function (hydrogen.py) and the file where you need to indicate the statistical characteristics shared by the group of generated fields (hydrogen_input.txt) in the folder where this Jupyter Notebook is located. Further details on how to compile the .txt file can be found in the "Hydro_genInfo.pdf" and in the "manual_hydrogen.pdf". 

-2 on WINDOWS: the executable file must be in the  folder. the hydrogen function (hydrogen.py) and the file where you need to indicate the statistical characteristics shared by the group of generated fields (hydrogen_input.txt) in the folder where this Jupyter Notebook is located. The windows executalbe code is available at https://github.com/alberto-bellin/Hydro_gen

**OUTPUT**: the output of this section is a .npy file (Kfileds_Hydrogen.npy) containing the $K$ values of each of the generated $K$-fields. Each column of the generated file contains the $K$-values of one of one of the filed field.....  (complete with Jinwoo)
If you want to proceed with all the following steps of the simulation on a Windows platform, just copy and past the "Kfileds_Hydrogen.npy" file create with the Linux or Mac platforms in the folder where this Jupyter Notebook is located, comment the entire kernel 3 and proceed with the simulation. 

In [ ]:
#default values 3.sigy: variance of the logconductivity field; 1.60943 cond10: mean of the logconductivity field

#Indicate the operating system that you are using to run this kernel
operating_system = 'win' # or 'mac' or 'linux'

#Generate the random K-fileds usig the previusly imported function
fields = hg.field_generation(N_mc, operating_system)

#Save all the generated fields in a unique file 
np.save('Kfileds_Hydrogen.npy', fields)

# restore the directory 
os.chdir(current_directory)


## 4. Flow Simulations
### FloPy

In this section we are going to simulate flow of groudwater through the just generated aquifers. To do that we use $\text{MODFLOW}$ that is the U.S. Geological Survey modular finite-difference flow model, which is a computer code that solves the groundwater flow equation.  To create, run, and post-process $\text{MODFLOW}$-based models the Python packege $\text{FloPy}$ is used, that we already imported in the library section.

**WHAT YOU NEED**: To run the following kernel you need to include the $\text{MODFLOW}$ executable (mf2005dbl.exe) in the folder where this Jupyter Notebook is located. 

**OUTPUT**: In the folder $\text{tmp}$, a .ftl (flow-transport link) file for each flow simulation is saved. The .ftl file includes flow information necessary to run the contaminat transport simulation.


Further details on all the packages used here and available in  $\text{MODFLOW}$ and consequesntly implemented in $\text{FloPy}$ are available respectively at https://water.usgs.gov/ogw/modflow/MODFLOW-2005-Guide/ and https://flopy.readthedocs.io/en/3.3.2/index.html. 

In [ ]:
#Loop to solve flow simulations on the generated K-filds, for Monte Carlo analysis
for value in range(N_mc): 
    
    # Load K-field generated with Hydro_gen
    y_field = np.load('Kfileds_Hydrogen.npy', allow_pickle=True)[value]-1 # -1 is used to translate the field form mu  =1 to mu = 0
    
    modflow_exe = r'C:\Users\Andrea\Documents\PT\mf2005dbl.exe'

    # Hydraulic head difference along the x-direction
    delta_h = Lx

    # Name of the ftl output file, a different model for each generated K-filed
    outftl_name  = 'model-{}.ftl'.format(value) 

    # Init flopy and Create the output directory
    model_name = "example_Frontiers"

    if not os.path.exists('tmp'):
        os.mkdir('tmp')
    model_ws = "tmp"

    # MODFLOW model definition
    mf = flopy.modflow.Modflow(model_name, model_ws = model_ws, exe_name = modflow_exe)

    # Geometric variables for the DIS package 
    ztop = 0.
    zbot = np.zeros((nlay, nrow, ncol))
    for i in range(nlay):
        zbot[i, :, :] = ztop - del_L*(i+1)

    # Add DIS package to the MODFLOW model
    dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr = del_R, delc = del_C,
                               top = ztop, botm = zbot, perlen = 1)

    # Variables for the BAS package
    ibound = np.ones((nlay, nrow, ncol), dtype = np.int32)
    ibound[:, :,  0] = -1
    ibound[:, :, -1] = -1

    strt = np.zeros((nlay, nrow, ncol), dtype = np.float32)
    for i in range(nlay):
        for j in range(nrow):
            strt[i, j, :] = np.linspace(delta_h, 0, num = ncol)
    
    # Add BAS package to the MODFLOW model 
    bas = flopy.modflow.ModflowBas(mf, ibound = ibound, strt = strt)

    # Add LPF package to the MODFLOW model
    lpf = flopy.modflow.ModflowLpf(mf, hk = np.exp(y_field), layvka = 1, vka = 10)

    # Add OC package to the MODFLOW model
    oc = flopy.modflow.ModflowOc(mf)

    # Add PCG package to the MODFLOW model
    pcg = flopy.modflow.ModflowPcg(mf, mxiter = 500, iter1 = 300)

    # Add LMT package to the MODFLOW model
    lmt = flopy.modflow.ModflowLmt(mf, output_file_header = 'extended',
                                       output_file_format = 'formatted',
                                       output_file_name = outftl_name)

    # Write the MODFLOW model input files
    mf.write_input()

    # Run the MODFLOW model
    success, buff = mf.run_model()

    if not success:
        print('FLOW SIMULATION ERROR, SOMETHING WENT WRONG!')

In [ ]:
y_field_flip = y_field[::-1]

plt.imshow(y_field_flip,extent=[0, Lx, 0, Ly] )
# Add a colorbar
plt.colorbar(label="log(k)")

## 5. Contaminant Transport Simulations 
### $\mathbf{PAR^2}$



In this section we are going to simulate contamiant transport through the just generated aquifers using FloPy outputs. To do that we use $\text{PAR}^2$ that is a Lagrangian solute transport simulator that uses a parallelized Random Walk Particle Tracking (RWPT) method (https://github.com/GerryR/par2).

**WHAT YOU NEED**: To run the following kernel you need to include the $\text{PAR}^2$ executable (par2.exe) and the two .yaml files necessary respectively for definying the model variables (config.yaml) and for implementing the Monte Carlo analysis (config-tmp.yaml), all in the folder where this Jupyter Notebook is located. Additional informations on how to compile and define the variables of the contaminat transport simulation can be found on "PAR2Info.pdf". 

to run the code that stop whe the  first particle reach the control palane code FastestPath.py in the folder of this code 

**OUTPUT**: In the folder $\text{output}$, a .csv file (result-\*.csv) for each transport simulation is saved. This file includes the data related to the cumulative breakthrough curves at control planes of interest. For each transport simulation, snaphshot files can be generated at different time steps, indicating the location of the contamiant plume at the give time steps (snap-{}-\*.csv).  


In [ ]:
import FastestPath as FP

N_mc = N_mc

In [ ]:
folder_path = r"C:\Users\Andrea\Documents\PT\output"

# remove all the exixting output
# Iterate over all files in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the path points to a file
    if os.path.isfile(file_path):
        # Delete the file
        os.remove(file_path)
        print(f"Deleted file: {file_path}")
        
for value in range(N_mc): 
    
    # PAR2 executable
    par2_exe = 'par2.exe'

    # YAML Configuration file, transport simulation parameters can be modified here
    config_file = 'TransportSimulation/config.yaml'

    # YAML Configuration file, modification of config.yaml to implement the Monte Carlo Anlysis 
    configout = 'config-tmp.yaml'


    with open(config_file) as fin:
        with open(configout, 'w') as fout:
            for lineS in fin:
                fout.write(lineS.replace('{}', str(value)))

        print("EXECUTE PAR2 WITH FIELD {}".format(value))
    
    # Print the parameters in the configuration file
    with open(config_file, 'r') as stream:
        try:
            params = yaml.load(stream, Loader=yaml.FullLoader)
            print(yaml.dump(params))
        except yaml.YAMLError as exc:
            print(exc)

    # Create the output directory
    if not os.path.exists('output'):
        os.mkdir('output')

    print()

    # Run PAR2
    print('STARTING SIMULATION...')
    # Run the process and monitor the output folder
    FP.run_process(par2_exe, configout, value, xPlane)
    
    # Load the the matrix to smple the permability values  
    y_field = np.load('Kfileds_Hydrogen.npy', allow_pickle=True)[value]-1  # -1 is used to translate the field form mu  =1 to mu = 0
       
    y_field_flip = y_field[::-1]
    
#     plt.imshow(y_field_flip,extent=[0, Lx, 0, Ly] )
#     # Add a colorbar
#     plt.colorbar(label="log(k)")
    
    
    outPath = f"output/mHRpath-{value}.csv"
    path = pd.read_csv(outPath)
    
    grid_extents = [0, Lx, 0, Ly]
    resolution = (del_R, del_C)
    
    #inPath = f"output/mHRpath-{value}.csv"
    #path = pd.read_csv(inPath)
    
    pointsPath = list(zip(path['x'], path['y']))
    for pt in range(len(pointsPath)-1):
        list2 = FP.find_intersecting_points(pointsPath[pt], pointsPath[pt+1], grid_extents, resolution)
        if pt == 0:
            pathGrid = [pointsPath[pt]]
            pathGrid.extend(list2)
        else:
            pathGrid.extend([pointsPath[pt]])
            pathGrid.extend(list2)
    intermediate_points = FP.calculate_intermediate_points(pathGrid)
    # Sample grid values along the line
    sampled_values = FP.sample_grid_points(grid_extents, y_field, intermediate_points)
    # Remove valuse sampled 2 times 
    K_mHR_values = FP.remove_consecutive_duplicates(sampled_values)
    
    
    print("μ'",np.average(K_mHR_values))
    print('μ', np.average(y_field_flip))
    
    file_name = f"output/K_mHR_values-{value}.csv"
    
    
    
    with open(file_name, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(K_mHR_values)
        
    directory = r'C:\Users\Andrea\Documents\PT\output'
    for filename in os.listdir(directory): 
        if f"snap-{value-1}" in filename:
            rfile_path = os.path.join(directory, filename)
            os.remove(rfile_path)
            print(f"Deleted file: {rfile_path}")


#     # Plot a box on top of the plot in the coordinate of the source term
#     plt.plot([source_xl, source_xu, source_xu, source_xl,source_xl], [source_yl, source_yl, source_yu, source_yu,source_yl], "k", linewidth=2)
#     plt.plot(path['x'],path['y'], "r")
#     # Add a title and show the plot
#     plt.title("Hydraulic Conductivity")
#     plt.show()


In [ ]:
import os
import zipfile
from datetime import datetime

def create_zip_archive(source_folder, target_folder, config_folder,K_field_file ):
    # Create the target folder if it doesn't exist
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Get the current date and time
    current_datetime = datetime.now()
    date_time_str = current_datetime.strftime("%Y-%m-%d_%H-%M")

    # Create the zip file
    zip_filename = os.path.join(target_folder, f"{date_time_str}.zip")
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for foldername, subfolders, filenames in os.walk(source_folder):
            for filename in filenames:
                file_path = os.path.join(foldername, filename)
                zipf.write(file_path, os.path.relpath(file_path, source_folder))
        
        file_path = os.path.join(config_folder, 'config.yaml')
        zipf.write(file_path, arcname='config.yaml')
        
        file_path = os.path.join(K_field_file, 'Kfileds_Hydrogen.npy')
        zipf.write(file_path,  arcname='Kfileds_Hydrogen.npy')        


if __name__ == "__main__":
    source_folder = r'C:\Users\Andrea\Documents\PT\output'  # Replace with the actual path to the source folder
    target_folder = r"D:\Andrea\PT\output"  # Replace with the desired path for the target folder
    config_file = r"C:\Users\Andrea\Documents\PT\TransportSimulation"
    K_field_file = r"C:\Users\Andrea\Documents\PT"
    

    create_zip_archive(source_folder, target_folder, config_file,K_field_file)
    print("Files have been copied and zipped successfully.")
